🌟 核心概念：什么是 U-Net？
想象你在画一幅画：

左半边（编码器）：你先眯起眼睛看物体，忽略细节，只抓主要特征（这是“压缩”过程，提取语义）。

右半边（解码器）：你睁大眼睛，把刚才看到的特征还原回原始尺寸，画出轮廓（这是“放大”过程，精确定位）。

中间的桥梁（跳跃连接）：为了不让细节在“眯眼”的过程中丢失，你时不时回头看一眼原物体的细节，把这些细节直接“拼”到现在的画上。

知识点 1：基础积木 —— DoubleConv (双层卷积)
U-Net 的每一层都不是单独的一个卷积，而是“成双成对”出现的。

🎓 老师讲解：

为什么要两次？ 做一次卷积能看到的范围（感受野）比较小，连续做两次能让模型“看”得更广一点，提取的特征更扎实。

参数含义：3, 1, 1 分别代表卷积核大小 3x3，步长 1，填充 1。这是一种保持尺寸不变的卷积写法（输入 100x100，输出还是 100x100），方便后续拼接。

In [1]:
import torch  # 语法：导入 PyTorch 库。作用：这是深度学习的核心库，提供张量（Tensor）操作和自动求导功能。
import torch.nn as nn  # 语法：导入 PyTorch 的神经网络模块并简写为 nn。作用：提供构建网络层（如卷积、全连接、损失函数）的类。
import torchvision.transforms.functional as TF  # 语法：导入视觉变换函数库。作用：通常用于图像处理，在 U-Net 中常用于裁剪或缩放图片以匹配尺寸（虽然在这个类里还没用到，但在整个网络中会用）。

class DoubleConv(nn.Module):  # 语法：定义一个名为 DoubleConv 的类，继承自 nn.Module。作用：nn.Module 是所有神经网络模块的基类，继承它才能利用 PyTorch 的自动求导和参数管理功能。
    def __init__(self, in_channels, out_channels):  # 语法：定义构造函数（初始化方法），接收输入通道数和输出通道数。作用：在创建对象时执行，用于定义网络层的结构和参数。
        super().__init__()  # 语法：调用父类 (nn.Module) 的构造函数。作用：必须执行这一步，以初始化内部的模块注册机制，否则网络无法识别其中的参数。
        
        self.conv = nn.Sequential(  # 语法：创建一个 nn.Sequential 容器并赋值给 self.conv。作用：将多个网络层按顺序打包在一起，前一层的输出自动成为下一层的输入，简化 forward 的写法。
            
            # --- 第一层卷积块 ---
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),  
            # 语法：定义 2D 卷积层。参数：输入通道=in_channels，输出通道=out_channels，卷积核=3x3，步长=1，填充=1。
            # 作用：提取图像特征。
            # [关键点] padding=1 配合 kernel_size=3 可以保持特征图的长宽不变（Same Padding）。
            # [关键点] bias=False 是因为后面紧接着 BatchNorm，BN 会抵消偏置的作用，所以设为 False 节省参数。
            
            nn.BatchNorm2d(out_channels),  
            # 语法：定义 2D 批归一化层，输入维度必须等于上一层的 out_channels。
            # 作用：对数据进行归一化（减均值除方差），防止梯度消失/爆炸，加快训练收敛速度。
            
            nn.ReLU(),  
            # 语法：定义 ReLU 激活函数。inplace=True 表示直接在原内存上修改数据。
            # 作用：引入非线性（因为卷积是线性的），让神经网络能拟合复杂的特征。inplace=True 可以节省显存。

            # --- 第二层卷积块 ---
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),  
            # 语法：定义第二个 2D 卷积层。注意输入和输出都是 out_channels。
            # 作用：进一步提取更高层次的特征，增加网络深度（U-Net 的设计思想是每个阶段做两次卷积）。
            
            nn.BatchNorm2d(out_channels),  
            # 语法：第二个批归一化层。
            # 作用：同上，稳定数据分布。
            
            nn.ReLU(),  
            # 语法：第二个激活函数。
            # 作用：同上，增加非线性。
        )

    def forward(self, x):  # 语法：定义前向传播函数，x 是输入数据（张量）。作用：定义数据在网络中是如何流动的，调用 model(x) 时会自动执行此函数。
        return self.conv(x)  # 语法：将输入 x 传入 self.conv 容器。作用：x 会依次经过上面定义的 Conv -> BN -> ReLU -> Conv -> BN -> ReLU，并返回最终结果。

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        # 语法：继承 nn.Module 的标准写法，初始化父类
        # 作用：让你的类具备 PyTorch 神经网络的所有功能（如自动求导、保存权重）
        super().__init__()
        
        # 语法：实例化 nn.ModuleList，这是一个类似 Python list 的容器
        # 作用：创建“上采样（解码器）”层的列表。必须用 ModuleList 而不是普通 list，否则 PyTorch 无法识别里面的层，参数就不会被训练
        self.ups = nn.ModuleList()
        
        # 语法：实例化 nn.ModuleList
        # 作用：创建“下采样（编码器）”层的列表
        self.downs = nn.ModuleList()
        
        # 语法：定义最大池化层，核大小2，步长2
        # 作用：用于下采样，将图片的长宽减半（例如 256x256 -> 128x128），保留最显著的特征
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # === 1. 构建下采样部分 (Encoder) ===
        # 语法：遍历 features 列表 [64, 128, 256, 512]
        for feature in features:
            # 语法：向 ModuleList 中添加一个 DoubleConv 模块
            # 作用：添加一层卷积块。第一次循环：3->64；第二次：64->128...
            self.downs.append(DoubleConv(in_channels, feature))
            
            # 语法：更新变量
            # 作用：将当前输出通道数设为下一次的输入通道数（为下一层做准备）
            in_channels = feature

        # === 2. 构建上采样部分 (Decoder) ===
        # 语法：反向遍历 features 列表 [512, 256, 128, 64]
        # 作用：U-Net 右边是自底向上，所以要反着来
        for feature in reversed(features):
            # 步骤 A：先放一个转置卷积（用于放大图片）
            self.ups.append(
                nn.ConvTranspose2d(
                    # 输入通道是 feature*2，因为瓶颈层输出了 feature*2
                    feature*2, 
                    feature, 
                    kernel_size=2, stride=2, # 步长2意味着尺寸放大2倍
                )
            )
            # 步骤 B：再放一个双层卷积（用于融合特征）
            # 作用：输入是 feature*2，因为我们要把“跳跃连接过来的特征”和“刚刚放大的特征”拼在一起（拼接后通道数翻倍）
            self.ups.append(DoubleConv(feature*2, feature))

        # === 3. 构建瓶颈层 (Bottleneck) ===
        # 语法：定义最底层的双层卷积
        # 作用：U-Net 的最底部，连接编码器和解码器。输入 512 -> 输出 1024 (features[-1]*2)
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        
        # === 4. 输出层 ===
        # 语法：1x1 卷积
        # 作用：将特征图映射到我们需要的类别数(out_channels)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        # 语法：定义一个空列表
        # 作用：用于存储编码器每一层的输出，也就是“跳跃连接（Skip Connections）”需要的小抄
        skip_connections = []

        # === 前向传播：下采样过程 ===
        for down in self.downs:
            x = down(x)            # 通过双层卷积提取特征
            skip_connections.append(x) # 【关键】在池化之前，把特征图存起来！
            x = self.pool(x)       # 然后再池化，尺寸减半

        # === 前向传播：瓶颈层 ===
        x = self.bottleneck(x)     # 通过最底部
        
        # 语法：列表切片，[::-1] 表示倒序
        # 作用：因为解码器是从底向上的，所以我们需要把保存的跳跃连接也倒过来，让它们一一对应
        skip_connections = skip_connections[::-1]   #就是把列表翻转

        # === 前向传播：上采样过程 ===
        # 语法：步长为2的循环 (0, 2, 4...)
        # 作用：因为我们在 __init__ 里每次循环加了两个层（转置卷积+双层卷积），所以这里每次要处理两个层
        for idx in range(0, len(self.ups), 2):
            # 步骤 A：放大图片
            # self.ups[idx] 是 ConvTranspose2d
            x = self.ups[idx](x)
            
            # 步骤 B：获取对应的跳跃连接
            # idx//2 算出来分别是 0, 1, 2, 3，刚好对应倒序后的 skip_connections
            skip_connection = skip_connections[idx//2]

            # 步骤 C：尺寸对齐（处理输入尺寸不是16倍数的情况）
            if x.shape != skip_connection.shape:
                # 语法：使用 torchvision 的 resize 函数
                # 作用：如果放大后的 x 和保存的 skip_connection 尺寸对不上（比如 160 vs 161），强行把 x 拉伸成 skip_connection 的大小
                x = TF.resize(x, size=skip_connection.shape[2:])

            # 步骤 D：拼接
            # 语法：torch.cat，dim=1 表示在通道维度拼接
            # 作用：将“浅层的位置信息(skip_connection)”和“深层的语义信息(x)”结合在一起
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            # 步骤 E：特征融合
            # self.ups[idx+1] 是 DoubleConv
            x = self.ups[idx+1](concat_skip)

        # === 输出 ===
        return self.final_conv(x)

知识点 2：下采样 —— 越走越深（Encoder）
这是 U-Net 的左半边，“U”的下坡路。
📝 代码对应：
1. 初始化部分
self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # 池化层，让图片变小

2. Forward 部分
for down in self.downs:
    x = down(x)            # 1. 先做双层卷积提取特征
    skip_connections.append(x) # 2. 【关键】把这一层的特征存起来（为了待会拼接）
    x = self.pool(x)       # 3. 图片尺寸减半 (例如 160 -> 80)
3. 🎓 老师讲解：
目的：“舍弃空间，换取特征”。

通过 MaxPool，图片的长宽不断减半（160 -> 80 -> 40 -> 20），空间信息丢失了。

但是，通道数（channels）通常会翻倍（64 -> 128 -> 256...），意味着模型虽然看不清“哪里”有东西，但非常清楚图里“是什么”东西。

存起来 (append)：这里把还没有被缩小的 x 存进 skip_connections 列表，这是给右半边留的“小抄”。

知识点 3：瓶颈层 —— Bottleneck
这是 U-Net 的最底部，也是整个网络最“浓缩”的地方。

📝 代码对应：

```Python
self.bottleneck = DoubleConv(features[-1], features[-1]*2)
```

Forward 部分
x = self.bottleneck(x)
🎓 老师讲解：

此时图片尺寸最小（比如只剩 10x10 像素），但通道数最大（比如 1024）。

这里包含了图像最高级的语义信息（比如：“这是一只猫”，而不是“这里有一条白线”）

知识点 4：上采样与跳跃连接 —— 核心灵魂（Decoder）
这是 U-Net 的右半边，“U”的上坡路。也是新手最容易晕的地方。

📝 代码对应：

```Python
初始化部分：ups 列表里存的是 [转置卷积, 双层卷积, 转置卷积, 双层卷积...]
self.ups.append(nn.ConvTranspose2d(...)) 
self.ups.append(DoubleConv(...))

# Forward 部分
for idx in range(0, len(self.ups), 2):
    x = self.ups[idx](x)  # 1. 放大图片 (Upsample / TransposeConv)
    
    skip_connection = skip_connections[idx//2] # 2. 取出左边对应层的“小抄”

    # ... (中间省略了尺寸调整代码，知识点5讲) ...

    # 3. 【核心】拼接 (Concatenate)
    concat_skip = torch.cat((skip_connection, x), dim=1) 
    
    # 4. 融合特征
    x = self.ups[idx+1](concat_skip)
```
🎓 老师讲解：

放大 (ConvTranspose2d)：把缩小的图变大（20 -> 40）。但单纯放大后的图很模糊，丢失了边缘细节。

拼接 (torch.cat)：这是 U-Net 的灵魂！

ResNet 是把特征相加（值变大）。

U-Net 是把特征拼接（通道数变多）。

意义：左边存的 skip_connection 包含了清晰的位置/纹理信息，右边放大的 x 包含了语义信息。把它们拼在一起，模型就能既知道“是什么”，又知道“在哪里”。

知识点 5：尺寸对齐 —— 解决报错的补丁
你给的代码中有一段看似多余的 if 判断，其实是为了增强模型的鲁棒性。

📝 代码对应：

Python
if x.shape != skip_connection.shape:
    # 调整 x 的尺寸，让它和 skip_connection 一样大
    x = TF.resize(x, size=skip_connection.shape[2:])
🎓 老师讲解：

为什么要这一步？ 看看你的测试案例：输入是 161 x 161。

左边下采样：161 除以 2，在 PyTorch 池化中通常向下取整，变成 80。

右边上采样：80 乘以 2，变成 160。

冲突：左边的“小抄”是 161，右边放大回来是 160。161 != 160，如果不处理，执行 torch.cat 拼接时程序就会崩溃。

解决办法：在拼接前，强行把右边的图 resize（或 crop）成和左边一样大。代码中使用 TF.resize 完美解决了这个问题，让 U-Net 可以处理任意尺寸的输入（甚至是奇数尺寸）。

In [3]:
x = torch.randn((3, 1, 161, 161))
model = UNET(in_channels=1, out_channels=1)
preds = model(x)
assert preds.shape == x.shape

In [ ]:
import torch  # [语法]: 导入 PyTorch 深度学习核心库
import torch.nn as nn  # [语法]: 导入神经网络模块 (Neural Network)
import torch.optim as optim  # [语法]: 导入优化器模块 (Optimizer)
from torch.utils.data import DataLoader  # [语法]: 导入数据加载器，用于批量读取数据
import torchvision  # [语法]: 导入 PyTorch 的计算机视觉库 (包含数据集、模型、变换等)
import torchvision.transforms as transforms  # [语法]: 导入图像预处理/变换工具
from torchvision.transforms import InterpolationMode  # [语法]: 导入插值模式枚举类
from tqdm import tqdm  # [语法]: 导入进度条库，用于显示训练进度

# ==========================================
# 1. 超参数设置 (Hyperparameters)
# ==========================================
LEARNING_RATE = 1e-4  
# [语法]: 浮点数赋值
# [作用]: 学习率。决定了梯度下降时更新参数的步长。1e-4 (0.0001) 是微调网络常用的保守值。

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# [语法]: 三元运算符 (条件表达式)
# [作用]: 自动检测设备。如果显卡 (NVIDIA GPU) 可用则使用 "cuda"，否则使用 "cpu"。

BATCH_SIZE = 8 
# [语法]: 整数赋值
# [作用]: 批大小。一次训练喂给神经网络多少张图片。显存越小，这个数要越小 (如 8 或 4)。

NUM_EPOCHS = 10
# [语法]: 整数赋值
# [作用]: 训练轮数。整个数据集被模型遍历学习多少次。

NUM_WORKERS = 2
# [语法]: 整数赋值
# [作用]: 数据加载的工作进程数。设置为 2 表示用 2 个 CPU 核心并行预读取数据，加快速度。

IMAGE_HEIGHT = 320 
IMAGE_WIDTH = 480
# [语法]: 整数赋值
# [作用]: 统一输入尺寸。神经网络要求同一个 Batch 内的图片大小必须一致，所以要强制缩放。

NUM_CLASSES = 21 
# [语法]: 整数赋值
# [作用]: 分类数量。PASCAL VOC 数据集包含 20 种物体 + 1 种背景 = 21 类。

# ==========================================
# 2. 定义数据预处理
# ==========================================
# 输入图片处理 (Training Images)
img_transform = transforms.Compose([
    # [语法]: 组合变换操作
    # [作用]: 将多个预处理步骤串联起来执行。
    
    transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
    # [语法]: 调整图像大小
    # [作用]: 将所有输入图片强制缩放到 320x480，以便打包成 Batch。默认使用双线性插值 (Bilinear)。
    
    transforms.ToTensor(),
    # [语法]: 转换为张量
    # [作用]: 将 PIL 图片 (0-255 整数) 转换为 PyTorch Tensor (0.0-1.0 浮点数)，并把通道移到前面 (HWC -> CHW)。
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # [语法]: 标准化
    # [作用]: 使用 ImageNet 的均值和方差对图片进行归一化。这能加速模型收敛，是迁移学习的标准操作。
])

# 标签处理 (Segmentation Masks)
label_transform = transforms.Compose([
    transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=InterpolationMode.NEAREST),
    # [语法]: 调整大小，强制使用最近邻插值 (NEAREST)
    # [作用]: ★关键点★！标签图里的像素值代表类别 (0=背景, 1=飞机...)。
    #        如果用默认的双线性插值，1 和 2 之间会算出 1.5，这不是有效的类别 ID。
    #        最近邻插值保证像素值要么是 1，要么是 2，不会产生中间值。
    
    transforms.PILToTensor(),
    # [语法]: PIL 转 Tensor (但不除以 255)
    # [作用]: 我们需要保持标签里的整数类别 ID (0, 1, 2...)，而不是把它们变成 0.0-1.0 的小数。
    #        注意这里不能用 ToTensor()，因为 ToTensor() 会自动除以 255。
])

# ==========================================
# 3. 准备数据集
# ==========================================
print("正在加载 VOC 数据集...")
ROOT_DIR = 'D:\\深度学习\\200_李沐数据集\\data' 
# [语法]: 字符串赋值
# [作用]: 定义数据集在硬盘上的根目录路径。

# 训练集对象
train_dataset = torchvision.datasets.VOCSegmentation(
    root=ROOT_DIR,         # [参数]: 数据集根目录
    year='2012',           # [参数]: 使用 VOC 2012 版本
    image_set='train',     # [参数]: 读取 'train.txt' 中定义的训练集图片
    download=False,        # [参数]: 是否自动下载 (如果已下载设为 False)
    transform=img_transform,       # [参数]: 对原始图片应用上面定义的 img_transform
    target_transform=label_transform # [参数]: 对标签图片应用上面定义的 label_transform
)

# 验证集对象
val_dataset = torchvision.datasets.VOCSegmentation(
    root=ROOT_DIR, 
    year='2012', 
    image_set='val',       # [参数]: 读取 'val.txt' 中定义的验证集图片
    download=False, 
    transform=img_transform, 
    target_transform=label_transform
)

# 训练数据加载器
train_loader = DataLoader(
    train_dataset,         # [参数]: 数据集对象
    batch_size=BATCH_SIZE, # [参数]: 每次读取多少张图 (16)
    shuffle=True,          # [参数]: 是否打乱顺序。训练集必须打乱，防止模型记顺序。
    num_workers=NUM_WORKERS, # [参数]: 多进程读取
    drop_last=True         # [参数]: 如果最后剩下的图片不足一个 Batch，就丢弃 (防止 Tensor 形状不一致报错)。
)

# 验证数据加载器
val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,         # [参数]: 验证集不需要打乱
    num_workers=NUM_WORKERS, 
    drop_last=True
)

# ==========================================
# 4. 训练函数
# ==========================================
# ==========================================
# 4. 训练函数 (朴素版 - 无混合精度)
# ==========================================
# [语法]: 定义函数 train_fn
# [作用]: 封装一个 Epoch (一轮) 内的训练逻辑。
# 参数 loader: 数据加载器，负责源源不断地吐出 Batch 数据。
# 参数 model: 你的 U-Net 神经网络模型。
# 参数 optimizer: 优化器 (如 Adam)，负责根据梯度修改模型参数。
# 参数 loss_fn: 损失函数 (如 CrossEntropy)，负责给模型的表现打分。
def train_fn(loader, model, optimizer, loss_fn): 
    
    # [语法]: tqdm 包装器
    # [作用]: 给 loader 加一个进度条。desc="Training" 会显示在进度条前面，让你知道现在是在训练阶段。
    loop = tqdm(loader, desc="Training")

    # [语法]: for 循环 + enumerate 枚举
    # [作用]: 遍历整个数据集。
    # batch_idx: 当前是第几个 Batch (0, 1, 2...)。
    # (data, targets): 解包出来的元组。data 是图片，targets 是标签掩膜。
    for batch_idx, (data, targets) in enumerate(loop):
        
        # [语法]: Tensor.to() 方法
        # [作用]: 将输入图片搬运到 GPU 显存上 (如果 DEVICE='cuda')。
        # ★必须★：模型在 GPU 上，数据也必须在 GPU 上才能运算。
        data = data.to(DEVICE)
        
        # [语法]: 链式调用 .to().squeeze().long()
        # [作用]: 处理标签数据。
        # 1. .to(DEVICE): 搬运到 GPU。
        # 2. .squeeze(1): 压缩维度。原始 targets 是 [Batch, 1, H, W]，交叉熵损失函数需要 [Batch, H, W]，所以要把中间那个 1 去掉。
        # 3. .long(): 类型转换。将数据转为 64位整数 (int64)。分类任务的标签必须是整数，不能是浮点数。
        targets = targets.to(DEVICE).squeeze(1).long()

        # -------------------------------------------
        # 1. 前向传播 (Forward)
        # -------------------------------------------
        
        # [语法]: 调用模型实例 (本质是调用 model.forward(data))
        # [作用]: 将图片喂给 U-Net，计算出预测结果。
        # predictions 的形状通常是 [Batch, 21, H, W] (21 是类别数)。
        predictions = model(data)
        
        # [语法]: 调用损失函数实例
        # [作用]: 计算“预测值”和“真实值”之间的差距。差距越大，loss 越大。
        # 这一步会构建计算图，为反向传播做准备。
        loss = loss_fn(predictions, targets)

        # -------------------------------------------
        # 2. 反向传播 (Backward)
        # -------------------------------------------
        
        # [语法]: 优化器的 .zero_grad() 方法
        # [作用]: 梯度清零。
        # PyTorch 设计上默认会累加梯度（为了某些特殊任务）。在常规训练中，必须在反向传播前清空上一步的梯度，否则梯度会乱掉。
        optimizer.zero_grad() 
        
        # [语法]: 损失张量的 .backward() 方法
        # [作用]: 反向传播 (Backpropagation)。
        # 根据链式法则，自动计算 Loss 对网络中每一个权重参数的导数 (梯度)。
        # 此时，参数的 .grad 属性里就有了具体的数值。
        loss.backward()       
        
        # [语法]: 优化器的 .step() 方法
        # [作用]: 参数更新。
        # 优化器利用刚才算出来的梯度，按照学习率 (lr) 稍微修改一下模型的权重，让模型变得更聪明一点点。
        # 公式大致为：weight = weight - lr * gradient
        optimizer.step()      

        # [语法]: tqdm 对象的 .set_postfix() 方法
        # [作用]: 更新进度条的后缀信息。
        # loss.item(): 取出 loss 张量里的数值 (float)。
        # 效果：进度条后面会实时跳动显示 loss=0.xxx，让你直观看到训练有没有收敛。
        loop.set_postfix(loss=loss.item())

# ==========================================
# 5. 主程序
# ==========================================
model = UNET(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    train_fn(train_loader, model, optimizer, loss_fn) 



正在加载 VOC 数据集...

Epoch 1/10


Training:  14%|█▍        | 26/183 [06:32<36:32, 13.96s/it, loss=2.71] 